## 2.5. Automatic Differentiation 自动微分
在上一节中说道, 求导是几乎所有深度学习优化算法的关键步骤。 虽然求导的计算很简单，只需要一些基本的微积分。 但对于复杂的模型，手工进行更新是一件很痛苦的事情（而且经常容易出错）。
深度学习框架通过自动计算导数，即自动微分（automatic differentiation）来加快求导。 实际中，根据设计好的模型，系统会构建一个计算图（computational graph）， 来跟踪计算是哪些数据通过哪些操作组合起来产生输出。 自动微分使系统能够随后反向传播梯度。 这里，反向传播（backpropagate）意味着跟踪整个计算图，填充关于每个参数的偏导数。 

- 自动微分主要有两种模式：
  - 前向模式自动微分（Forward Mode AD）：从输入开始逐步计算到输出。
  - 反向模式自动微分（Reverse Mode AD）：从输出开始逐步计算到输入，这种模式更适用于深度学习中的多层网络，因为计算效率更高。

- 计算图（Computational Graph）是一个有向无环图（DAG），表示计算过程中的操作和变量。在计算图中：
  - 节点（Nodes）表示变量或中间计算结果。
  - 边（Edges）表示变量之间的操作。

- 通过计算图，我们可以清晰地表示计算的依赖关系，并系统化地追踪计算过程。例如，对于一个简单的函数 $f(x) = x^2+yx$，我们可以画出计算图如下：
    $$x \rightarrow [平方] \rightarrow x^2 \\ y \rightarrow [乘法] \rightarrow yx \\ [加法] \rightarrow x^2 + yx$$
  
- 为什么反向传播梯度重要
  - 高效计算：反向传播算法能够高效地计算复杂模型的梯度，避免了手工推导的繁琐和容易出错。
  - 优化参数：在深度学习中，梯度用于优化模型参数，通过最小化损失函数来提高模型的性能。反向传播使得我们能够系统化地计算梯度，从而优化模型。
  - 自动化流程：自动微分和反向传播允许深度学习框架自动处理梯度计算，使得模型训练和调试变得更加方便和高效。

为了详细说明计算图如何使用反向传播计算梯度，我们将以 $y = 2\mathbf{x}^T \mathbf{x}$ 为例来演示。这个过程包括前向传播（计算函数值）和反向传播（计算梯度）的步骤。我们将逐步构建计算图，并展示如何通过这个图计算梯度。

**前向传播**
函数: $y = 2\mathbf{x}^T \mathbf{x}$; 假设 $\mathbf{x} = \begin{pmatrix} x_1 \\ x_2 \\ x_3 \\ x_4 \end{pmatrix}$，即 $\mathbf{x}$ 是一个四维向量。
步骤:
1. 计算 $\mathbf{x}^T \mathbf{x}$：向量的点积。
2. 将结果乘以 2。

构建计算图如下：
```
   x_1    x_2    x_3    x_4
    |      |      |      |
    |      |      |      |
  [平方] [平方] [平方] [平方]
    |      |      |      |
    +------|------|------+
           |
        [加法]
           |
           v
          S = x_1^2 + x_2^2 + x_3^2 + x_4^2
           |
        [乘以2]
           |
           v
          y = 2 * S
```

**反向传播,计算梯度**

目标: 计算 $\nabla_{\mathbf{x}} y$，即 $y$ 关于 $\mathbf{x}$ 的梯度。在反向传播过程中，我们从输出节点开始，逐层计算梯度，直到输入节点。具体步骤如下：
1. 从最终输出 $y$ 开始, $y = 2S$, 
这里 $S = \mathbf{x}^T \mathbf{x}$
所以我们有： $\frac{\partial y}{\partial S} = 2$

2. 传播到 $S$: $S = x_1^2 + x_2^2 + x_3^2 + x_4^2$, 我们需要计算 $\frac{\partial S}{\partial x_i}$
$\frac{\partial S}{\partial x_i} = 2x_i$   对于每个   $i \in \{1, 2, 3, 4\}$

3. 应用链式法则 $\frac{\partial y}{\partial x_i} = \frac{\partial y}{\partial S} \cdot \frac{\partial S}{\partial x_i}$
因此, $\frac{\partial y}{\partial x_i} = 2 \cdot 2x_i = 4x_i$

4. 所以, $\nabla_{\mathbf{x}} y = \begin{pmatrix} 4x_1 \\ 4x_2 \\ 4x_3 \\ 4x_4 \end{pmatrix}=4\mathbf{x}$

### 2.5.1. A Simple Function 一个简单的例子
作为一个演示例子，假设我们想对函数 $y = 2\mathbf{x}^T \mathbf{x}$ 关于列向量 $\mathbf{x}$ 求导。 根据公式$\nabla_{\mathbf{x}} \mathbf{x}^\top \mathbf{x} = 2\mathbf{x}$我们可以很容易地计算导数。 结果应该是 $4\mathbf{x}$。

In [63]:
from mxnet import autograd, np, npx

npx.set_np()

x = np.arange(4.0)  # 创建一个包含 [0.0, 1.0, 2.0, 3.0] 的一维数组（向量）
x

array([0., 1., 2., 3.])

在我们计算$y$关于$\mathbf{x}$的导数之前，需要一个地方来存储梯度。 重要的是，我们不会在每次对一个参数求导时都分配新的内存。 
因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。 
注意，一个标量函数关于向量$\mathbf{x}$的梯度是向量，并且与$\mathbf{x}$具有相同的形状。在数学中，标量函数（scalar function）是指输入为向量或标量，但输出为单个标量的函数。 这里，函数$y = 2\mathbf{x}^T \mathbf{x}$的输入是一个向量$\mathbf{x}$，输出是一个标量。

举例说明, 为什么一个标量函数关于向量$\mathbf{x}$的梯度是向量，并且与$\mathbf{x}$具有相同的形状; 考虑标量函数 $f(\mathbf{x}) = \mathbf{x}^T \mathbf{x}$，其中 $\mathbf{x} = \begin{pmatrix} x_1 \\ x_2 \end{pmatrix}$ 是一个二维向量。

1. 函数展开：$f(\mathbf{x}) = x_1^2 + x_2^2$
2. 计算偏导数：$\frac{\partial f}{\partial x_1} = 2x_1, \quad \frac{\partial f}{\partial x_2} = 2x_2$
3. 梯度向量：$\nabla_{\mathbf{x}} f(\mathbf{x}) = \begin{pmatrix} 2x_1 \\ 2x_2 \end{pmatrix}$

可见梯度向量 $\nabla_{\mathbf{x}} f(\mathbf{x})$ 的维度与输入向量 $\mathbf{x}$ 相同，都是二维向量。


In [64]:
# 通过调用attach_grad来为一个张量的梯度分配内存
x.attach_grad()  # 将x的梯度分配内存, 且与x形状相同
# 在计算关于x的梯度后，将能够通过'grad'属性访问它，它的值被初始化为0
x.grad

array([0., 0., 0., 0.])

现在计算 $y$
在代码中，y = 2 * np.dot(x, x) 实际上与数学表达式 $y = 2\mathbf{x}^T \mathbf{x}$ 是等价的。 
这是因为在 NumPy 中，np.dot(x, x) 对于一维数组（向量）来说，计算的结果是内积（dot product），而这个内积在数学上等价于 $\mathbf{x}^T \mathbf{x}$。

In [65]:
# 把代码放到autograd.record内，以建立计算图
with autograd.record():  # 记录计算图。在这个 with 上下文管理器内部执行的所有操作都会被记录下来，从而允许后续进行反向传播计算梯度。
    y = 2 * np.dot(x, x)
x, y

(array([0., 1., 2., 3.]), array(28.))

x是一个长度为4的向量，计算x和x的点积，得到了我们赋值给y的标量输出。 接下来，通过调用反向传播函数来自动计算y关于x每个分量的梯度，并打印这些梯度。

In [66]:
y.backward()  # 反向传播计算梯度
x.grad

array([ 0.,  4.,  8., 12.])

函数 $y = 2\mathbf{x}^T \mathbf{x}$ 关于列向量 $\mathbf{x}$ 的梯度应该是 $4\mathbf{x}$。 让我们快速验证这个梯度是否计算正确。

In [67]:
x.grad == 4 * x

array([ True,  True,  True,  True])

现在计算x的另一个函数。

In [68]:
with autograd.record():
    y = x.sum()
y.backward()
x.grad  # 被新计算的梯度覆盖

array([1., 1., 1., 1.])

### 2.5.2. Backward for Non-Scalar Variables 非标量变量的反向传播
当$y$不是标量时，向量$\mathbf{y}$关于向量$\mathbf{x}$的导数的最自然解释是一个矩阵。 

向量$\mathbf{y}$关于向量$\mathbf{x}$的导数: 当输出 $\mathbf{y}$ 和输入 $\mathbf{x}$ 都是向量时，计算它们的导数（也称为雅可比矩阵）自然会得到一个矩阵。对于 $\mathbf{y} \in \mathbb{R}^m$ 和 $\mathbf{x} \in \mathbb{R}^n$，雅可比矩阵 $ J$ 的大小是 $ m \times n$，其元素是 $\frac{\partial y_i}{\partial x_j}$。

对于高阶和高维的$\mathbf{y}$和$\mathbf{x}$，求导的结果可以是一个高阶张量。例如, 当输入和输出都是二维矩阵时，导数的结果可能会是一个三维张量。对于更高阶和高维的情况，导数可能会变得更加复杂。

然而，虽然这些更奇特的对象确实出现在高级机器学习中（包括深度学习中）， 但当调用向量的反向计算时，我们通常会试图计算一批训练样本中每个组成部分的损失函数的导数。 
这里，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。

例子解释
- 损失函数与批量处理
  - 假设我们有一个损失函数 $L$ 和一批训练样本。对于每个样本 $i$，损失函数为 $L_i$，我们要计算的是批量中所有样本损失的总和的梯度，即：$L_{\text{batch}} = \sum_{i} L_i$
- 梯度计算
  - 我们计算的是总损失函数 $L_{\text{batch}}$ 对模型参数 $\mathbf{x}$ 的梯度，而不是每个单独的 $\frac{\partial L_i}{\partial \mathbf{x}}$。在这种情况下，我们需要计算： 
    $$\nabla_{\mathbf{x}} L_{\text{batch}} = \nabla_{\mathbf{x}} \left( \sum_{i} L_i \right) = \sum_{i} \nabla_{\mathbf{x}} L_i$$
- 为什么这样做
  - 计算效率：计算一个总梯度比计算雅可比矩阵要简单得多，尤其是在高维情况下。
  - 实际需求：在模型训练中，我们关心的是如何调整模型参数以最小化总损失函数，而不是单个样本的损失。
  - 内存和计算资源：计算并存储高阶张量或雅可比矩阵需要大量的内存和计算资源。通过只计算总梯度，我们可以节省这些资源。

In [69]:
x, x * x

(array([0., 1., 2., 3.]), array([0., 1., 4., 9.]))

In [70]:
# 当对向量值变量y（关于x的函数）调用backward时，将通过对y中的元素求和来创建一个新的标量变量。然后计算这个标量变量相对于x的梯度
with autograd.record():
    y = x * x  # y是一个向量
y.backward()  # 方向传播
x.grad  # 查看梯度, 等价于y=sum(x*x)

array([0., 2., 4., 6.])

为了理解如何通过计算图计算梯度，让我们分解上面代码的每个步骤：

1. 前向传播：
- 计算 $ y = x \cdot x $, 这里，$y = [x_1^2, x_2^2, x_3^2, x_4^2]$：
2. 隐式求和：
- 调用 `y.backward()` 时，系统会隐式地将 `y` 中的元素求和：$L = \sum_{i} y_i = x_1^2 + x_2^2 + x_3^2 + x_4^2$
3. 计算梯度：
- 反向传播计算标量 $L$ 关于向量 $x$ 的梯度：$\frac{\partial L}{\partial x_i} = \frac{\partial (x_1^2 + x_2^2 + x_3^2 + x_4^2)}{\partial x_i} = 2x_i$
因此，梯度向量是：$\nabla_{\mathbf{x}} L = \begin{pmatrix} 2x_1 \\ 2x_2 \\ 2x_3 \\ 2x_4 \end{pmatrix}$

**为什么梯度是向量而不是矩阵**
- 当我们调用 `y.backward()` 时，我们在计算一个标量（$L$）关于向量（$\mathbf{x}$）的导数。结果是一个向量，而不是矩阵。
- 这个向量是 $L$ 关于 $\mathbf{x}$ 的梯度，而不是 $L$ 中每个元素关于 $\mathbf{x}$ 的梯度。
- 如果我们没有对 $\mathbf{y}$ 的元素求和，而是对每个元素分别计算梯度，那么梯度将是一个矩阵（雅可比矩阵）。但是在深度学习中，我们通常需要的是损失函数（标量）关于输入参数的梯度，这就是为什么最终的梯度是一个与 $\mathbf{x}$ 形状相同的向量。

## 2.5.3. Detaching Computation 分离计算
有时，我们希望将某些计算移动到记录的计算图之外。 例如，假设y是作为x的函数计算的，而z则是作为y和x的函数计算的。 想象一下，我们想计算z关于x的梯度，但由于某种原因，希望将y视为一个常数， 并且只考虑到x在y被计算后发挥的作用。
这里可以分离y来返回一个新变量u，该变量与y具有相同的值， 但丢弃计算图中如何计算y的任何信息。 换句话说，梯度不会向后流经u到x。 因此，下面的反向传播函数计算z=u*x关于x的偏导数，同时将u作为常数处理， 而不是z=x*x*x关于x的偏导数。

例子解释
假设我们有变量 $x$：
- $y$ 是 $x$ 的函数：$y = f(x)$
- $z$ 是 $y$ 和 $x$ 的函数：$z = g(y, x)$

通常情况下，当我们计算 $z$ 关于 $x$ 的梯度时，梯度会通过 $y$ 传播回 $x$，即考虑了 $y$ 作为 $x$ 的函数的影响。然而，有时我们希望将 $y$ 视为常数，不考虑 $y$ 的计算过程对 $x$ 的影响。
这可以通过分离计算来实现。通过分离计算，我们创建一个新变量 $u$，它的值等于 $y$，但丢弃了如何计算 $y$ 的任何信息。这样，梯度在反向传播时不会通过 $u$ 向前传播到 $x$。

**正常的计算图**

1. 前向传播：
   $$
   y = x^2
   $$
   $$
   z = y \cdot x = x^2 \cdot x = x^3
   $$

2. 反向传播：
   - 正常情况下，计算 $z$ 关于 $x$ 的梯度会考虑到 $y$ 的计算过程：
     $$
     \frac{dz}{dx} = \frac{d}{dx}(x^3) = 3x^2
     $$

**分离计算图**

1. 前向传播：
   - 创建一个新变量 $u$，使 $u$ 等于 $y$ 的值，但不包含计算 $y$ 的信息：$$u = y.detach() \quad \text{（假设这里的 detach 操作分离了计算图）}$$
   - 现在计算 $z$：
     $$
     z = u \cdot x
     $$
2. 反向传播：
   - 由于 $u$ 是从 $y$ 分离出来的，梯度不会通过 $u$ 传播到 $x$，因此我们只考虑 $z$ 关于 $x$ 的直接影响：$$
     \frac{dz}{dx} = \frac{d}{dx}(u \cdot x) = u
     $$
   - 这里 $u$ 等于 $y$，即 $y = x^2$，所以：
     $$
     \frac{dz}{dx} = x^2
     $$
   - 当我们计算 $z = u \cdot x$ 关于 $x$ 的偏导数时，如果我们将 $u$ 视为常量，那么梯度就是 $u$: $   \frac{\partial z}{\partial x} = \frac{\partial (u \cdot x)}{\partial x} = u$
      

In [71]:
with autograd.record():
    y = x * x
    u = y.detach()
    z = u * x
z.backward()
x.grad, x.grad == u

(array([0., 1., 4., 9.]), array([ True,  True,  True,  True]))

由于记录了y的计算结果，我们可以随后在y上调用反向传播， 得到y=x*x关于的x的导数，即2*x。

In [72]:
y.backward()  # 在这个步骤中，MXNet 计算了 y = x*x 关于 x 的梯度
x.grad, x.grad == 2 * x  # y对于x的梯度, 就是2x

(array([0., 2., 4., 6.]), array([ True,  True,  True,  True]))

## 2.5.4. Gradients and Python Control Flow Python控制流的梯度计算
使用自动微分的一个好处是： 即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度。 在下面的代码中，while循环的迭代次数和if语句的结果都取决于输入a的值。

In [73]:
def f(a):
    b = a * 2
    while np.linalg.norm(b) < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

# 让我们计算梯度。
a = np.random.normal()
a.attach_grad()
with autograd.record():
    d = f(a)
d.backward()
a.grad 

array(102400.)